## Project details: Mapping Tabulis muris data to scMuscle_Cosgrove_atlas
#### Query1: Limb Muscle
#### Query2: Bladder (-ve control)

## Set up

In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px

#### Replace path with path/to/repo

In [ ]:
projpath="/Users/hinabandukwala/Desktop/Bioinformatics/DILWORTH_LAB/Data_analyzed/Cosgrove_muscle_atlas_2021/scMuscle_AtlasMapping"

## Original metadata and umap embeddings 

##### from scMuscle_diet_v1-1.h5Seurat

In [ ]:

Orig_UMAP_coords = pd.read_csv(os.path.join(projpath, "data/scMuscle_diet_data/UMAP_embeddings.csv"), index_col=0)

Orig_Metadata = pd.read_csv(os.path.join(projpath, "data/scMuscle_diet_data/scMusclediet_metadata.csv"), index_col=0)

In [ ]:
Metadata_UMAP = pd.concat([Orig_UMAP_coords, Orig_Metadata], axis=1, join="inner")
Metadata_UMAP.head()

In [ ]:
ColorDict = {"Myonuclei (Type IIb)" : "lightcoral", 
            "Myonuclei (Type IIx)" : "firebrick",
            "Myoblasts/Progenitors" : "orangered",
            "MuSCs" : "magenta",
            
            "NK Cells" : "tan",
            "T Cells" : "goldenrod",
            "Neutrophils" : "yellow",
             "B Cells" : "bisque",
            
            "Monocyte (Inflammatory)" : "olivedrab",
            "Monocyte (Patrolling)" : "palegreen",
            "M2 Macro. (Cx3cr1_hi)" : "darkgreen",
            "M2 Macro. (Cx3cr1_lo)": "mediumturquoise",
            "Monocyte (Cxcl10+)": "darkkhaki",
            "Dendritic": "khaki",
             
            "Endothelial (Capillary)" : "deepskyblue",
            "Endothelial (Artery)": "royalblue",
            "Endothelial (Vein)" : "navy",
            "Smooth Muscle & Pericytes": "slateblue",
            "Neural": "darkviolet",
            
            "FAPs (Adipogenic)": "powderblue",
            "Tenocytes" : "orchid",
            "FAPs (Pro-remodeling)": "blueviolet",
            "FAPs (Stem)": "crimson"}

##### Static plot showing scMucsle_umap

In [ ]:
p = sns.scatterplot(x="UMAPHarmony_1", y="UMAPHarmony_2", 
                    data=Metadata_UMAP, hue="harmony_factorIDs", palette=ColorDict)

sns.set_style("whitegrid", {'axes.grid' : False})



plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

##### Interactive plot for scMuscle_umap

In [ ]:
fig = px.scatter(x=Metadata_UMAP["UMAPHarmony_1"], y=Metadata_UMAP["UMAPHarmony_2"], 
                 color=Metadata_UMAP['harmony_factorIDs'],
                 color_discrete_map=ColorDict)
                

fig.update_traces(marker=dict(size=12,
                              opacity=1,
                              line=dict(width=0.5,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
                              




fig.update_layout(legend=dict(
     font=dict(
        family="Courier New, monospace",
        size=18)))


fig.show()

## UMAP after recalculating embeddings

In [ ]:
# Using results from the first query mapping just bc it contains the recalculated umap as well
TM = pd.read_csv(os.path.join(projpath, "results/TM_Mus_MapQueryResults.csv"))
TM = TM.set_index('Unnamed: 0')

In [ ]:
TM['celltype'] = TM['cell_type_pred_knn']

TM["celltype"].fillna(TM["harmony_factorIDs"], inplace=True)


Ref = TM.loc[TM['ref_query'] == 'reference']
Ref.shape

In [ ]:
fig = px.scatter(x=Ref["UMAP1"], y=Ref["UMAP2"], 
                 color=Ref['harmony_factorIDs'],
                color_discrete_map=ColorDict)



fig.update_traces(marker=dict(size=12,
                              opacity=1,
                              line=dict(width=0.5,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.update_layout(legend=dict(
     font=dict(
        family="Courier New, monospace",
        size=18)))


fig.show()

##### Compare the new Ref umap to the original

In [ ]:
# create figures
fig, axes = plt.subplots(1, 2, figsize=(8, 8))
fig.suptitle('Reference UMAP')

# plot data
p1 = sns.scatterplot(ax=axes[0], x="UMAPHarmony_1", y="UMAPHarmony_2", 
                    data=Metadata_UMAP, hue="harmony_factorIDs", palette=ColorDict, legend=False)
p1.set(ylabel=None)  # remove the axis label


p2 = sns.scatterplot(ax=axes[1], x="UMAP1", y="UMAP2", 
                    data=Ref, hue="celltype", palette=ColorDict, legend=False)
p2.set(ylabel=None)  # remove the axis label


sns.set_style("whitegrid", {'axes.grid' : False})


#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


## Query 1: Limb Muscle 

In [ ]:
Limb_Mus_annotation = pd.read_csv(os.path.join(projpath, "data/TabulaMuris_data/Limb_Muscle_droplet_annotation.csv" ))
Limb_Mus_annotation = Limb_Mus_annotation.set_index('cell')
Limb_Mus_annotation = Limb_Mus_annotation[['cell_ontology_class', 'tSNE_1' ,'tSNE_2']]
Limb_Mus_annotation.index = Limb_Mus_annotation.index + "-1"


TM = TM[['UMAP1', 'UMAP2', 'cell_type_pred_knn']]

Plotting_DF = pd.concat([TM, Limb_Mus_annotation], axis=1, join="inner")
Plotting_DF['cell_ontology_class'] = Plotting_DF['cell_ontology_class'].fillna("None")

In [ ]:
px.scatter(Plotting_DF, x="tSNE_1", y="tSNE_2", color="cell_type_pred_knn")

In [ ]:
px.scatter(Plotting_DF, x="tSNE_1", y="tSNE_2", color="cell_ontology_class")

In [ ]:
px.scatter(Plotting_DF, x="UMAP1", y="UMAP2", color="cell_type_pred_knn")

### How many cells were identified correctly vs incorrectly?

In [ ]:
Plotting_DF 

In [ ]:
Plotting_DF['cell_type_pred_knn'].unique()

In [ ]:
Plotting_DF['cell_ontology_class'].unique()

In [ ]:
IdentDict = {'Endothelial (Vein)': 'endothelial cell',
            'Endothelial (Capillary)' : 'endothelial cell',
           'Endothelial (Artery)' : 'endothelial cell',
            'T Cells' : 'T cell',
            'B Cells' : 'B cell',
            'MuSCs' : 'skeletal muscle satellite cell',
            'macrophage' : 'M2 Macro. (Cx3cr1_hi)',
            'FAPs (Adipogenic)' : 'mesenchymal stem cell',
            'FAPs (Pro-remodeling)' : 'mesenchymal stem cell',
            'FAPs (Stem)' : 'mesenchymal stem cell'}

In [ ]:
Plotting_DF['CorrMapping'] = ""
for cell in Plotting_DF.index:
    Mapping = Plotting_DF.loc[cell]['cell_type_pred_knn']
    Original = Plotting_DF.loc[cell]['cell_ontology_class']
    if Mapping in IdentDict.keys():
        if IdentDict[Mapping] == Original:
            Plotting_DF.loc[cell, 'CorrMapping'] = True
        else:
            Plotting_DF.loc[cell, 'CorrMapping'] = False
    else:
        Plotting_DF.loc[cell, 'CorrMapping'] = False
    
        
    
        

In [ ]:
identdf = Plotting_DF[["cell_type_pred_knn", "CorrMapping"]]



In [ ]:

chart = sns.catplot(data=identdf,kind='count',x='cell_type_pred_knn',hue='CorrMapping',
                   height=8.27, aspect=11.7/8.27)
chart.set_xticklabels(rotation=90, horizontalalignment='right')



## Query 2: Bladder

In [ ]:
Bladder_annotation = pd.read_csv(os.path.join(projpath, "data/TabulaMuris_data/Bladder_droplet_annotation.csv" ))
Bladder_annotation = Bladder_annotation.set_index('cell')
Bladder_annotation = Bladder_annotation[['cell_ontology_class', 'tSNE_1' ,'tSNE_2']]
Bladder_annotation.index = Bladder_annotation.index + "-1"


TM = pd.read_csv(os.path.join(projpath, "results/TM_Bladder_MapQueryResults.csv"))
TM = TM[["Unnamed: 0", 'UMAP1', 'UMAP2', 'cell_type_pred_knn']]
TM = TM.set_index('Unnamed: 0')

Plotting_DF = pd.concat([TM, Bladder_annotation], axis=1, join="inner")

In [ ]:
px.scatter(Plotting_DF, x="tSNE_1", y="tSNE_2", color="cell_type_pred_knn")

In [ ]:
px.scatter(Plotting_DF, x="tSNE_1", y="tSNE_2", color="cell_ontology_class")

In [ ]:
px.scatter(Plotting_DF, x="UMAP1", y="UMAP2", color="cell_type_pred_knn")

### How many were identified correctly?

In [ ]:
Plotting_DF

In [ ]:
Plotting_DF['CorrMapping'] = ""
for cell in Plotting_DF.index:
    Mapping = Plotting_DF.loc[cell]['cell_type_pred_knn']
    Original = Plotting_DF.loc[cell]['cell_ontology_class']
    if Mapping in IdentDict.keys():
        if IdentDict[Mapping] == Original:
            Plotting_DF.loc[cell, 'CorrMapping'] = True
        else:
            Plotting_DF.loc[cell, 'CorrMapping'] = False
    else:
        Plotting_DF.loc[cell, 'CorrMapping'] = False

In [ ]:
Plotting_DF

In [ ]:
chart = sns.catplot(data=Plotting_DF,kind='count',x='cell_type_pred_knn',hue='CorrMapping',
                   height=8.27, aspect=11.7/8.27)
chart.set_xticklabels(rotation=90, horizontalalignment='right')